In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SNOBAL_DIR, DATA_DIR, SNOTEL_DIR, COARSEN_OPTS, use_hvplot, HV_PLOT_OPTS

use_hvplot()

In [ ]:
client = start_cluster(6, 24)
client_ip_and_port(client)

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'

year = 2021
water_year = f'wy{year}'

## SNOTEL 

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites_2x2.json')

schofield_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Schofield/usda-csv/{year}-Schofield-Pass.csv',
)

butte_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Butte/usda-csv/{year}-Butte.csv',
)

taylor_snotel_csv = CsvParser.file(
    SNOTEL_DIR / water_year / f'Taylor/usda-csv/{year}-Upper-Taylor.csv',
)

## Model

In [ ]:
solar_HRRR = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw_hrrr_solar/run*/snow.nc',
    parallel=True,
)

butte_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).compute()
schofield_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).compute()
taylor_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).compute()

In [ ]:
solar_SMRF = xr.open_mfdataset(
    f'{SNOBAL_DIR}/GMD/{water_year}/erw/run*/snow.nc',
    parallel=True,
)

butte_snobal = solar_SMRF.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat).compute()
schofield_snobal = solar_SMRF.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat).compute()
taylor_snobal = solar_SMRF.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat).compute()

# SNOTEL 

## Butte 

In [ ]:
butte_snobal_hrrr.thickness.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).hvplot(label='HRRR').opts(**HV_PLOT_OPTS) * \
butte_snobal.thickness.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).hvplot(label='SMRF') * \
butte_snotel_csv['Depth(m)'].plot(label='SNOTEL')

## Schofield Pass 

In [ ]:
schofield_snobal_hrrr.thickness.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).hvplot(label='HRRR solar').opts(**HV_PLOT_OPTS) * \
schofield_snobal.thickness.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).hvplot(label='SMRF solar') * \
schofield_snotel_csv['Depth(m)'].plot(label='SNOTEL')

## Upper Taylor 

In [ ]:
taylor_snobal_hrrr.thickness.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).hvplot(label='HRRR').opts(**HV_PLOT_OPTS) * \
taylor_snobal.thickness.coarsen(**COARSEN_OPTS).mean().squeeze(['x', 'y']).hvplot(label='SMRF') * \
taylor_snotel_csv['Depth(m)'].plot(label='SNOTEL')